# CS 178 Project
## Import part

In [1]:
import os
import numpy as np
import pandas as pd
import cv2

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Read image for train

In [2]:
data = pd.read_csv('data/500_picts_satz.csv')


image_paths = data.iloc[:, 1]
labels = data.iloc[:, 2]

folder_path = 'images/'

X = []
y = labels

for filename in image_paths:
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # Check for common image file extensions
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        # img - cv2.resize(img, (350, 350))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img is not None:
            X.append(torch.Tensor(img))
X = torch.stack(X).unsqueeze(1) # convert to pytorch tensor
# X = torch.stack(X) # convert to pytorch tensor
# (batch, channels, w, h)
print(X.shape)

# Xtr, ytr = X[:300], y[:300]
# Xva, yva = X[300:500], y[300:500]
seed = 1234
Xtr, ytr, Xva, yva = train_test_split(X, y, test_size=0.25, random_state=seed)

torch.Size([499, 1, 350, 350])


## CNN Test
### Define a Convolutional Neural Network

In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5) # 1 because its greyscale (in_channels, out_channels, kernel_size)
        print(self.conv1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        print(self.conv2)
        self.fc1 = nn.Linear(16 * 21 * 21, 120) # shape of image after pooling twice
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 7) # 7 is label count

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))


### Define a Loss function and optimizer

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Train the network

In [13]:
batch_size = 2
trainloader = torch.utils.data.DataLoader(Xtr, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(ytr, batch_size=batch_size, shuffle=False, num_workers=2)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(testloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        print(outputs.shape)
        print(labels.shape)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

torch.Size([16, 7])
torch.Size([1, 350, 350])


ValueError: Expected input batch_size (16) to match target batch_size (1).